In [119]:
import json
from os.path import expanduser
db = expanduser("~/Desktop/reduced_dblp.json")
database = json.loads(open(db, 'r').read())
G = nx.Graph()

In [120]:
# DAATABASE = List of PUBBLICATION
# PUBBLICATION -> 6 DICT= authors, id_conference, id_conference_int, id_publication, id_publication_int, title
# AUTHORS -> List of DICT (About 10500 in reduced json) -> {(author:Name , author_id:ID)}

In [121]:
import networkx as nx
import itertools
from collections import defaultdict

def pubblicationDictionary(json):
    pubb_dict = {}
    for pubblication in json:
        for author in pubblication["authors"]:
            if "&" in author["author"]:
                pass
            else:
                try:
                    pubb_dict[(author["author"],author["author_id"])] += [(pubblication["id_publication_int"],pubblication["id_publication"])]
                except:
                    pubb_dict[(author["author"],author["author_id"])] = [(pubblication["id_publication_int"],pubblication["id_publication"])]
    return(pubb_dict)


dictAutor = pubblicationDictionary(database)

for j in dictAutor.keys():
    G.add_node(j[1], id = j[1], author=j[0])

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 7406
Number of edges: 0
Average degree:   0.0000


In [123]:
dict_publ = {}
for elem in range(len(database)):
    d = database[elem]['authors']
    for author in range(len(d)):
        if "&" in d[author]['author']:
            pass
        else:
            try:
                dict_publ[database[elem]['id_publication']].append((d[author]['author'], d[author]['author_id']))
            except:
                dict_publ[database[elem]['id_publication']] = [(d[author]['author'], d[author]['author_id'])]

dew = defaultdict(list)
for keys in dictAutor.keys():
    for i in range(len(dictAutor[keys])):
        try:
            dew[keys].append(dictAutor[keys][i][0])
        except:
            dew[keys]=dictAutor[keys][i][0]

def Jaccard(a,b):
    jaccard = (abs(len(list(set(a)|set(b))))-abs(len(list(set(a)&set(b)))))/abs(len(list(set(a)|set(b))))
    return jaccard

# ADD EDGE            
for k,v in dict_publ.items():
    for i in itertools.combinations(v,2):
        G.add_edge(i[0][1],i[1][1], pubblication=k[0], pubblication_int = k[1], weight=Jaccard(dew[i[0]],dew[i[1]]))

print(nx.info(G))


Name: 
Type: Graph
Number of nodes: 7406
Number of edges: 15368
Average degree:   4.1501


In [124]:
elarge=[(u,v) for (u,v,d) in G.edges(data=True) if d['weight'] == 0.5]
print(len(elarge))

3001
